# Crack identification procedure

In [ ]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from bmcs_shear.dic_crack import\
    DICGrid, DICStateFields, DICCrackList, \
    DICAlignedGrid
import numpy as np
np.seterr(divide ='ignore', invalid='ignore');
import collections
collections.Iterable = collections.abc.Iterable

In [ ]:
dic_grid = DICGrid(U_factor=100, dir_name='B6_TV1', t=1)

In [ ]:
dic_grid.read_beam_design()

In [ ]:
from bmcs_shear.dic_crack import DICStressProfile
dsf = DICStateFields(dic_grid=dic_grid, R=7, omega_threshold=0.15)
l_cb = dic_grid.d_x * 1
dsf.tmodel_.trait_set(E=5000, c_T=0, nu=0.18, epsilon_0=0.0005, epsilon_f=0.02/l_cb, eps_max=0.005);
dsf.eval()
dcl = DICCrackList(dsf=dsf, corridor_left=3, corridor_right=1)
dcl.identify_cracks()

In [ ]:
dcl.interact()